<h1> Использование датасета Klarna для дообучения трансформера MarkupLM-base.

In [ ]:
# Подключаем Google-диск с загруженным датасетом

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Сначала будет код для распаковки датасета и его загрузки на Google-диск (его необязательно выполнять и даже вредно!).

Распаковываем датасет с английскими страницами.

In [ ]:
!unzip /content/drive/MyDrive/Klarna/klarna_dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/US/www.stevemadden.com/5534/source.html  
   creating: train/US/www.stevemadden.com/6919/
  inflating: train/US/www.stevemadden.com/6919/elements_metadata.json  
  inflating: train/US/www.stevemadden.com/6919/page_metadata.json  
  inflating: train/US/www.stevemadden.com/6919/source.html  
   creating: train/US/www.stevemadden.com/7683/
  inflating: train/US/www.stevemadden.com/7683/elements_metadata.json  
  inflating: train/US/www.stevemadden.com/7683/page_metadata.json  
  inflating: train/US/www.stevemadden.com/7683/source.html  
   creating: train/US/www.stevemadden.com/8782/
  inflating: train/US/www.stevemadden.com/8782/elements_metadata.json  
  inflating: train/US/www.stevemadden.com/8782/page_metadata.json  
  inflating: train/US/www.stevemadden.com/8782/source.html  
   creating: train/US/www.stevemadden.com/9459/
  inflating: train/US/www.stevemadden.com/9459/elements_metadata.json  
  in

Функция для подсчёта файлов в папке test/train. Используем её, чтобы оценить в дальнейшем размер обучающей и тестовой выборок.

In [ ]:
import os

def count_files(path: str) -> int:
    count = 0
    for country in os.listdir(path):
        for dir in os.listdir(f"{path}/{country}"):
            count += len(os.listdir(f"{path}/{country}/{dir}"))
    return count

In [ ]:
train_num = count_files("/content/train")
train_num

15428

In [ ]:
test_num = count_files("/content/test")
test_num

6719

Теперь реализуем функции для получения произвольных страниц из датасета.

In [ ]:
def get_all_files(path: str) -> list():
    dirs = list()
    for country in os.listdir(path):
        for website in os.listdir(f"{path}/{country}"):
            for filename in os.listdir(f"{path}/{country}/{website}"):
                dirs.append(f"{path}/{country}/{website}/{filename}")
    return dirs

Функцию, которая выберет нам случайное подмножество страниц.

In [ ]:
from random import choices


def get_train_test_files(dir: str, train_num: int, test_num: int) -> tuple[list, list]:
    train_files = get_all_files(f"{dir}/train")
    test_files = get_all_files(f"{dir}/test")
    return choices(train_files, k=train_num), choices(test_files, k=test_num)

Сейчас выберем случайное подмножество страниц для обучения и тестирования. Возьмём их в следующем соотношении: 8500 страниц для обучения и 1000 страниц для тестирования. Мы понимаем, что такие значения адекватные, потому что знаем общее число страниц; разделяем обучающую/тестовую выборку в соотношении ~ 0.9/0.1

In [ ]:
train_files, test_files = get_train_test_files("/content", 8500, 1000)

In [ ]:
train_files[:10]

['/content/train/US/eu.mophie.com/6570',
 '/content/train/GB/www.365games.co.uk/12656',
 '/content/train/GB/www.radley.co.uk/29623',
 '/content/train/GB/m.size.co.uk/35064',
 '/content/train/GB/www.rideworx.co.uk/31224',
 '/content/train/US/www.footlocker.com/11493',
 '/content/train/GB/www.trespass.com/20315',
 '/content/train/GB/www.fitness-superstore.co.uk/24447',
 '/content/train/US/buynailsdirect.com/7841',
 '/content/train/US/m.dickssportinggoods.com/15900']

In [ ]:
test_files[:10]

['/content/test/US/www.williams-sonoma.com/4783',
 '/content/test/GB/m.mybag.com/8587',
 '/content/test/GB/www.securitydoorcompany.com/13929',
 '/content/test/GB/www.titansoftoys.com/15334',
 '/content/test/GB/www.tweediechairs.co.uk/15174',
 '/content/test/GB/www.aldi.co.uk/6610',
 '/content/test/GB/www.leoofficesupplies.co.uk/13638',
 '/content/test/GB/www.abtecdirect.com/14704',
 '/content/test/GB/www.123spareparts.co.uk/5039',
 '/content/test/GB/www.weldricks.co.uk/9385']

Сохраним наши выборки для обучения и тестирования.

In [ ]:
import shutil

def copy_files(files: list, dst: str) -> None:
    for ind, obj in enumerate(files, start=1):
        shutil.copy(f"{obj}/source.html", f"{dst}/source{ind}.html")

In [ ]:
copy_files(train_files, "/content/drive/MyDrive/Klarna/train")

In [ ]:
copy_files(test_files, "/content/drive/MyDrive/Klarna/test")

Теперь мы можем приступить к основной части обучения.

P.S. Код здесь в большом количестве скопирован с предыдущего пункта задания, так что, возможно, некоторая его часть останется без объяснения (за таковым смотрите предыдущий ноутбук).

Сначала установим нужные библиотеки.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git  # Transormers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.0 MB/s eta 0:00:00


In [ ]:
!pip install -q evaluate seqeval  # HuggingFace metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.3 MB/s eta 0:00:00


In [ ]:
# Устанавливаем необходимую версию PyTorch, чтобы избежать ненужных ошибок

!pip uninstall torch -y
!pip install torch==2.0.0

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168

Сначала мы определим, что хотим извлекать со страниц. Этими атрибутами будут название товара и его цена:

In [ ]:
labels = ["name", "price", "other"]

labels_with_ids = {el: ind for ind, el in enumerate(labels)}
ids_with_labels = {v:k for k, v in labels_with_ids.items()}

In [ ]:
ids_with_labels

{0: 'name', 1: 'price', 2: 'other'}

Теперь организуем работу с данными. Для этого, как и в предыдущем ноутбуке, создадим специальный контейнер, предоставляемый нам PyTorch.

In [ ]:
from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx] # May be to delete!
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length",
                                  max_length=self.max_length, return_tensors="pt", truncation=True)

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

Для loss-функции нужны истинные значения. Найдём их с помощью следующих функций.

In [ ]:
from torch._C import NoneType
from bs4 import BeautifulSoup


# Функция для обработки полученного элемента из html
def get_label_text(obj: any) -> str:
    if not obj:
        return ""
    else:
        return obj.get_text()


# Истинные значения искомых атрибутов
def get_true_labels(html_page: str) -> dict:
    soup = BeautifulSoup(html_page)
    name = soup.find(attrs={"klarna-ai-label": "Name"})
    price = soup.find(attrs={"klarna-ai-label": "Price"})
    return {"name": get_label_text(name),
            "price": get_label_text(price)}

Здесь получаем данные из html-страниц для создания датасета для обучения/тестирования.

In [ ]:
from transformers import MarkupLMFeatureExtractor, MarkupLMProcessor


feature_extractor = MarkupLMFeatureExtractor()
processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

def get_data(files: list) -> list:
    data = []
    for page in files:
        with open(page, 'r') as html_page:
            html_code = html_page.read()
        true_labels = get_true_labels(html_code)
        encoding = feature_extractor(html_code)
        node_labels = [[]]
        for node_text in encoding['nodes'][0]:
            # do this with match! ( match <expr>: ) (Still did not realized)
            if node_text == true_labels["name"]:
                node_labels[0].append(labels_with_ids["name"])
            elif node_text == true_labels["price"]:
                node_labels[0].append(labels_with_ids["price"])
            else:
                node_labels[0].append(labels_with_ids["other"])
            
        data.append({'nodes': encoding['nodes'],
                    'xpaths': encoding['xpaths'],
                    'node_labels': node_labels})
    return data

Теперь можем и собрать нужные нам датасеты.

In [ ]:
import os

def get_files(dir: str) -> list:
    files = list()
    for filename in os.listdir(dir):
        files.append(f"{dir}/{filename}")
    return files

In [ ]:
from torch.utils.data import DataLoader

def get_dataloaders(train_files: list, test_files: list) -> tuple[DataLoader, DataLoader]:
    train_data = get_data(train_files)
    train_dataset = MarkupLMDataset(data=train_data, processor=processor, max_length=512)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    
    test_data = get_data(test_files)
    test_dataset = MarkupLMDataset(data=test_data, processor=processor, max_length=512)
    test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)
    return train_dataloader, test_dataloader

Следующая строчка кода будет выполняться долго (~1 часа), поэтому нам необходимо сохранить её с помощью модуля pickle.

In [ ]:
train_dataloader, test_dataloader = get_dataloaders(get_files("/content/drive/MyDrive/Klarna/train"),
                                                    get_files("/content/drive/MyDrive/Klarna/test"))

Функции для сохранения и загрузки данных из/в оперативную память ноутбука.

In [ ]:
import pickle


def save_data(object: any, dst: str) -> None:
    with open(dst, "wb") as f:
        pickle.dump(object, f, pickle.HIGHEST_PROTOCOL)


def load_data(filename: str) -> any:
    with open(filename, "rb") as f:
        return pickle.load(f)

Сохраняем данные на диск.

In [ ]:
save_data(train_dataloader, "/content/drive/MyDrive/Klarna/train.pickle")
save_data(test_dataloader, "/content/drive/MyDrive/Klarna/test.pickle")

In [ ]:
del train_dataloader
del test_dataloader

Теперь мы можем сразу получить преобразованный датасет! Загружаем его с диска.

In [ ]:
train_dataloader = load_data("/content/drive/MyDrive/Klarna/train.pickle")
test_dataloader = load_data("/content/drive/MyDrive/Klarna/test.pickle")

Дальше мы должны обучать модель. \
Для нашего удоства и не только нам нужно реализовать метрики, по которым можно судить модель. Они реализованы ниже.

In [ ]:
import evaluate


label_list = ["B-" + x for x in list(ids_with_labels.values())]


def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(metric, return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

Дальше реализованы функции обучения и тестирования модели.

In [ ]:
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
from pprint import PrettyPrinter


pp = PrettyPrinter(indent=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train_model(model, dataloader: DataLoader, epoch_num: int, start_epoch: int=0,
                print_epoch: bool=False, save_model: bool=False, dir: str=None) -> None:
    optimizer = AdamW(model.parameters(), lr=5e-5, betas=(0.9, 0.98), eps=1e-6, weight_decay=0.01)
    model.to(device)
    model.train()
    for epoch in range(start_epoch, epoch_num):  # loop over the dataset multiple times
        metric = evaluate.load("seqeval")
        for batch in dataloader:
            inputs = {k:v.to(device) for k,v in batch.items()}
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            predictions = outputs.logits.argmax(dim=-1)
            true_labels = batch["labels"]
            preds, refs = get_labels(predictions, true_labels)
            metric.add_batch(
                predictions=preds,
                references=refs,
            )

        eval_metric = compute_metrics(metric)
        if print_epoch:
            print(f"---Epoch {epoch + 1}:---")
            print("Loss:", loss.item())
            pp.pprint(eval_metric)
        if save_model:
            torch.save(model.state_dict(), f"{dir}/model_{epoch}")


In [ ]:
import gc


def test_model(model: any, dataloader: DataLoader) -> any:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    metric = evaluate.load("seqeval")
    for batch in dataloader:
        inputs = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**inputs)
        gc.collect()
        torch.cuda.empty_cache()
        predictions = outputs.logits.argmax(dim=-1)
        true_labels = batch["labels"]
        preds, refs = get_labels(predictions, true_labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )
    eval_metric = compute_metrics(metric)
    return eval_metric


Будем обучать одну модель, но будем это делать в 20 эпох.

In [ ]:
# First 5 epochs

from transformers import MarkupLMForTokenClassification
import gc


torch.cuda.empty_cache()
gc.collect()

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=ids_with_labels, label2id=labels_with_ids)
train_model(model, train_dataloader, epoch_num=20,
            print_epoch=True,
            save_model=True, dir="/content/drive/MyDrive/Klarna/models")

Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['cls.predictions.transform.dense.bias', 'markuplm.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'nrp_cls.LayerNorm.bias', 'ptc_cls.bias', 'cls.predictions.decoder.bias', 'markuplm.pooler.dense.bias', 'nrp_cls.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'nrp_cls.decoder.bias', 'ptc_cls.weight', 'nrp_cls.dense.bias', 'nrp_cls.decoder.weight', 'cls.predictions.bias', 'nrp_cls.LayerNorm.weight']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

---Epoch 1:---
Loss: 0.006618651561439037
{   'name_f1': 0.7770726996827149,
    'name_number': 7271,
    'name_precision': 0.7794382177943822,
    'name_recall': 0.7747214963553845,
    'other_f1': 0.9971055103077705,
    'other_number': 1038871,
    'other_precision': 0.99677460004752,
    'other_recall': 0.9974366403528446,
    'overall_accuracy': 0.9942608321317209,
    'overall_f1': 0.9942608321317209,
    'overall_precision': 0.9942608321317209,
    'overall_recall': 0.9942608321317209,
    'price_f1': 0.6009109023626529,
    'price_number': 3836,
    'price_precision': 0.6617554858934169,
    'price_recall': 0.5503128258602711}
---Epoch 2:---
Loss: 0.004937007557600737
{   'name_f1': 0.908713415042529,
    'name_number': 7271,
    'name_precision': 0.8856396866840731,
    'name_recall': 0.9330215926282492,
    'other_f1': 0.9987565308607088,
    'other_number': 1038871,
    'other_precision': 0.9989917005495088,
    'other_recall': 0.9985214718670556,
    'overall_accuracy': 0.9

In [ ]:
# Next 6-13 epochs

from transformers import MarkupLMForTokenClassification
import gc


torch.cuda.empty_cache()
gc.collect()

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=ids_with_labels, label2id=labels_with_ids)
model.load_state_dict(torch.load("/content/drive/MyDrive/Klarna/models/model_4"))
train_model(model, train_dataloader, epoch_num=20, start_epoch=5,
            print_epoch=True,
            save_model=True, dir="/content/drive/MyDrive/Klarna/models")

Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['cls.predictions.transform.dense.weight', 'nrp_cls.dense.bias', 'markuplm.pooler.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'nrp_cls.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'markuplm.pooler.dense.bias', 'nrp_cls.LayerNorm.weight', 'nrp_cls.LayerNorm.bias', 'ptc_cls.bias', 'nrp_cls.decoder.bias', 'nrp_cls.dense.weight', 'ptc_cls.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

---Epoch 6:---
Loss: 0.0010290147038176656
{   'name_f1': 0.9813963067206701,
    'name_number': 7271,
    'name_precision': 0.9797149122807017,
    'name_recall': 0.9830834823270527,
    'other_f1': 0.9997444332410053,
    'other_number': 1038871,
    'other_precision': 0.9997478014446549,
    'other_recall': 0.9997410650600508,
    'overall_accuracy': 0.9994933227172379,
    'overall_f1': 0.9994933227172379,
    'overall_precision': 0.9994933227172379,
    'overall_recall': 0.9994933227172379,
    'price_f1': 0.9657695322707082,
    'price_number': 3836,
    'price_precision': 0.9680460974332111,
    'price_recall': 0.9635036496350365}
---Epoch 7:---
Loss: 0.0028191597666591406
{   'name_f1': 0.9843320505772403,
    'name_number': 7271,
    'name_precision': 0.9836560911962643,
    'name_recall': 0.9850089396231605,
    'other_f1': 0.9997656124037723,
    'other_number': 1038871,
    'other_precision': 0.9997593571563331,
    'other_recall': 0.9997718677294871,
    'overall_accuracy'

In [ ]:
# Next 14-20 epochs

from transformers import MarkupLMForTokenClassification
import gc


torch.cuda.empty_cache()
gc.collect()

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=ids_with_labels, label2id=labels_with_ids)
model.load_state_dict(torch.load("/content/drive/MyDrive/Klarna/models/model_12"))
train_model(model, train_dataloader, epoch_num=20, start_epoch=13,
            print_epoch=True,
            save_model=True, dir="/content/drive/MyDrive/Klarna/models")

Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['ptc_cls.bias', 'cls.predictions.transform.dense.weight', 'nrp_cls.LayerNorm.weight', 'nrp_cls.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'markuplm.pooler.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'nrp_cls.decoder.bias', 'cls.predictions.decoder.weight', 'nrp_cls.decoder.weight', 'markuplm.pooler.dense.bias', 'nrp_cls.dense.weight', 'nrp_cls.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'ptc_cls.weight']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

---Epoch 14:---
Loss: 2.1400572222773917e-05
{   'name_f1': 0.9916035788024776,
    'name_number': 7271,
    'name_precision': 0.9924231987877118,
    'name_recall': 0.990785311511484,
    'other_f1': 0.9998878597332217,
    'other_number': 1038871,
    'other_precision': 0.999881603720916,
    'other_recall': 0.9998941158238126,
    'overall_accuracy': 0.9997780905885647,
    'overall_f1': 0.9997780905885647,
    'overall_precision': 0.9997780905885647,
    'overall_recall': 0.9997780905885647,
    'price_f1': 0.9855299178725069,
    'price_number': 3836,
    'price_precision': 0.9856584093872229,
    'price_recall': 0.9854014598540146}
---Epoch 15:---
Loss: 0.003327110316604376
{   'name_f1': 0.9938093272802311,
    'name_number': 7271,
    'name_precision': 0.9940828402366864,
    'name_recall': 0.993535964791638,
    'other_f1': 0.9999167368146972,
    'other_number': 1038871,
    'other_precision': 0.9999133680759434,
    'other_recall': 0.9999201055761495,
    'overall_accuracy':

Теперь протестируем нашу последнюю модель.

In [ ]:
# Скачиваем модель
from transformers import MarkupLMForTokenClassification
import gc

torch.cuda.empty_cache()
gc.collect()

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=ids_with_labels, label2id=labels_with_ids)
model.load_state_dict(torch.load("/content/drive/MyDrive/Klarna/models/model_19"))

Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['nrp_cls.LayerNorm.weight', 'nrp_cls.dense.bias', 'markuplm.pooler.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'nrp_cls.LayerNorm.bias', 'ptc_cls.bias', 'markuplm.pooler.dense.weight', 'nrp_cls.dense.weight', 'nrp_cls.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'ptc_cls.weight', 'cls.predictions.bias', 'nrp_cls.decoder.weight']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

<All keys matched successfully>

In [ ]:
# Тестируем


torch.cuda.empty_cache()
gc.collect()


del train_dataloader
test_metric = test_model(model, test_dataloader)
print(test_metric["overall_f1"])

0.9950604722910547


In [ ]:
test_metric

{'name_precision': 0.7744140625,
 'name_recall': 0.8743109151047409,
 'name_f1': 0.8213360952874159,
 'name_number': 907,
 'other_precision': 0.998471265609292,
 'other_recall': 0.9965338529789792,
 'other_f1': 0.9975016185527649,
 'other_number': 120595,
 'price_precision': 0.6175869120654397,
 'price_recall': 0.8118279569892473,
 'price_f1': 0.7015098722415797,
 'price_number': 372,
 'overall_precision': 0.9950604722910547,
 'overall_recall': 0.9950604722910547,
 'overall_f1': 0.9950604722910547,
 'overall_accuracy': 0.9950604722910547}

Итак, была обучена модель MarkupLM-base на датасете Klarna. Результаты выше.